<a href="https://colab.research.google.com/github/je-santos/MPLBM-UT/blob/master/examples/create_2D_training_data_4NN/VelNet_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vel-net
--------------
## Table of contents
- Libraries and examples
- Data visualization
- Data analysis
- Simplier methods: Linear regression
- Intro to NN: parabolic profile
- Intro to convnets

In [22]:
#@title Importing libraries and downloading data

!pip install git+https://github.com/je-santos/livelossplot

import os
import sys
import random

# Importing the numerical and viz libs
import numpy as np
import matplotlib.pyplot as plt

# Importing utilities libs 
import glob
from PIL import Image

# Importing the ML libs
import sklearn
import tensorflow as tf
from tensorflow import keras

from livelossplot import PlotLossesKeras
from livelossplot.plot_losses  import MatplotlibPlot
    
import warnings
warnings.simplefilter("ignore")


import gdown
url='https://drive.google.com/file/d/16m_6dpebJSSi9MAGRno4xKKFwV-Kq9oK'
try:
    os.mkdir('numpys')
except:
    print('The directory already exists')
output = 'numpys/numpys1.zip'
#gdown.download(url, output, quiet=False)
#!gdown --id url 

#from zipfile import ZipFile
#with ZipFile(output, 'r') as zipObj:
#   # Extract all the contents of zip file in current directory
#   zipObj.extractall()



  Cloning https://github.com/je-santos/livelossplot to c:\users\jae3287\appdata\local\temp\pip-req-build-4bgxpzmc
  Created wheel for livelossplot: filename=livelossplot-0.5.0-py3-none-any.whl size=20505 sha256=42abbeb8a3dfa6974d0ad5a28dccdeeb2d62fc6d69ec22d2c57acda9e15f083f
  Stored in directory: C:\Users\jae3287\AppData\Local\Temp\pip-ephem-wheel-cache-bgy90ve0\wheels\d1\85\7f\91c563589b9efae46dcb114d7983ea917ecaa1f37efb85023c
Successfully built livelossplot
